# Intereacting with the shell

## `sh` module

The [`sh` module](https://amoffat.github.io/sh/) is very convenient to interact with the shell.  Note that `sh` is not part of Python's standard library, if you prefer not to use extra modules, use the `subprocess` module in the standard library.

In [ ]:
import sh

Any shell command can be executed by calling it as a function on the `sh` module, passing command line arguments as arguments.

In [ ]:
sh.ls('-l')

The output can be used by assigning the command to a variable, and using the result's `stdout` attribute.  Note that the latter is a sequence of bytes, so it has to be decoded into a UTF-8 string for further processing.

In [ ]:
cmd = sh.ls('-l', '-a', _encoding='UTF-8')

In [ ]:
lines = cmd.stdout.decode(encoding='utf8').split('\n')

In [ ]:
len(lines[1:-1])

In [ ]:
lines[1:-1]

In [ ]:
_ = sh.mkdir('tmp', '-p')

In [ ]:
sh.ls()

### Exit codes

When a shell command fails, an exception is thrown which contains the full command as it was run, the exit code, the standard output and error.

In [ ]:
try:
    sh.ls('bla.txt')
except Exception as error:
    err_msg = error.stderr.decode(encoding='utf8').rstrip()
    print(f'command "{error.full_cmd}" exited with exit code {error.exit_code} and message "{err_msg}"')

### I/O redirection

Redirecting output can be done using the `_out` optional argument.

In [ ]:
with open('tmp/date_file.txt', 'w') as file:
    for i in range(10):
        print(f'{i} ', end='', file=file)
        sh.date(_out=file)
        sh.sleep('1')

In [ ]:
sh.cat('tmp/date_file.txt')

Input redirection works similarly using the optional `_in` argument.

In [ ]:
with open('tmp/date_file.txt', 'r') as file:
    print(sh.wc('-l', _in=file))

### Piping

The output of a command can be used as the input for another command.

Pipe the output of `ls` into `grep` to select only the files with names that end with `.py`.

In [ ]:
sh.grep(sh.ls('-l'), r'\.py$')

Pipe the output of `cut` into `sort`.

In [ ]:
sh.sort(sh.cut('-d', ' ', '-f', '5', 'tmp/date_file.txt'), '-r')

In [ ]:
_ = sh.rm('-r', 'tmp')

### Backgrounding & time out

Long running processes can be placed in the background.

In [ ]:
process = sh.sleep(10, _bg=True)

In [ ]:
for i in range(10):
    print(i)

In [ ]:
process.wait()

In [ ]:
print(process.exit_code)

A time out can be specified for a command, and on time out, the resulting exit code will be the number of the signal (SIGKILL by default).

In [ ]:
try:
    process = sh.sleep(10, _bg=True, _timeout=3)
except TimeoutError as error:
    print(error)

## `subprocess` module

If you prefer to use standard library modules only, `subprocess` is a good choice.

In [ ]:
import subprocess

This module has a high-level function `run` that can be used for almost any processing.  The API is still being improved in subsequent releases of Python.

In [ ]:
process = subprocess.run(['ls', '-l'], stdout=subprocess.PIPE, encoding='utf8')

In [ ]:
process.stdout.split('\n')

Note that if you don't specify the `stdout` arugment, the output of the command will not be captured.  Python 3.7 makes this easier by adding a `capture_output` argument.

### Exit codes

The `run` function returns a `CompletedProcess` object that has an attribute for the exit code returned by the process.

In [ ]:
process = subprocess.run(['mkdir', '-p', 'tmp'])

In [ ]:
process.returncode

### I/O redirection

Output of a running command can be redirected to a file.

In [ ]:
with open('tmp/data.txt', 'w') as file:
    for i in range(10):
        subprocess.run(['echo', '-n', str(i) + ' '], stdout=file)
        subprocess.run(['date'], stdout=file)
        subprocess.run(['sleep', '1'])

Note that mixed I/O from the `print` function and `run` doesn't work as expected.

In [ ]:
print(subprocess.run(['cat', 'tmp/data.txt'], stdout=subprocess.PIPE,
                     encoding='utf8').stdout)

Input redirection is similar.

In [ ]:
with open('tmp/data.txt', 'r') as file:
    process = subprocess.run(['wc', '-l'], stdin=file, stdout=subprocess.PIPE,
                             encoding='utf8')
    print(process.stdout)

### Piping

Piping can also be done using `subprocess`.  It is less user friendly than using the `sh` module, but it allows more control.  You will have to resort to the low-level `Popen` function.

In [ ]:
p1 = subprocess.Popen(['ls', '-l'], stdout=subprocess.PIPE)
p2 = subprocess.Popen(['grep', r'\.py$'], stdin=p1.stdout, stdout=subprocess.PIPE, encoding='utf8')
p1.stdout.close()
output, _ = p2.communicate()
print(output)

In [ ]:
p1 = subprocess.Popen(['cut', '-d', ' ', '-f', '5', 'tmp/data.txt'], stdout=subprocess.PIPE)
p2 = subprocess.Popen(['sort', '-r'], stdin=p1.stdout, stdout=subprocess.PIPE, encoding='utf8')
p1.stdout.close()
output, _ = p2.communicate()
print(output)

In [ ]:
_ = subprocess.run(['rm', '-r', 'tmp'])

### Shell file globbing and environment variables

In [ ]:
import os

For file globbing to work in subprocesses, provide the entire command, including all arguments as a string, rather than a list to `run`. Also, set `shell` to `True`.

In [ ]:
process = subprocess.run('ls *.py', stdout=subprocess.PIPE, encoding='utf8', shell=True)

In [ ]:
process.stdout.split()

The same applies when you want environment variables to expand.

In [ ]:
process = subprocess.run('echo "hello ${USER}"', stdout=subprocess.PIPE, encoding='utf8', shell=True)
print(process.stdout.rstrip())

If you need to add or modify environment variables, it is good practice to do that on a copy of `os.environ`.

In [ ]:
environ = os.environ.copy()
environ['greeting'] = 'bye'
process = subprocess.run('echo "${greeting} ${USER}"', stdout=subprocess.PIPE, encoding='utf8',
                         env=environ, shell=True)
print(process.stdout.rstrip())